<a href="https://colab.research.google.com/github/Habib0308/University_Final_Year_Project/blob/master/Notebooks/Chicago_Data_Preprocessing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objective of this preprocessing is to create encoded dataset for predicting Secondary Crime Description**


**Avoid colab clossing**
```
function ClickConnect(){
    document.querySelector("colab-connect-button").click()
    console.log("Clicked on connect button"); 
}
setInterval(ClickConnect,60000)
```

# **Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Importing Dataset**

In [ ]:
file_path= "/content/gdrive/My Drive/Final Year Project/Dataset/dataset_analyzed.csv"
data= pd.read_csv(file_path) #, nrows= 10000
df= data.copy()

# Changing Data type

In [ ]:
df["ward"]= df["ward"].astype(object)
df["beat"]= df["beat"].astype(object)
df["hour"]= df["hour"].astype(object)

In [ ]:
df.head(2)

,date_of_occurrence,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour
0,2019-06-20 05:02:00,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5
1,2019-06-20 05:15:00,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232015 entries, 0 to 232014
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date_of_occurrence     232015 non-null  object 
 1   block                  232015 non-null  object 
 2   iucr                   232015 non-null  object 
 3   primary_description    232015 non-null  object 
 4   secondary_description  232015 non-null  object 
 5   location_description   232015 non-null  object 
 6   arrest                 232015 non-null  object 
 7   domestic               232015 non-null  object 
 8   beat                   232015 non-null  object 
 9   ward                   232015 non-null  object 
 10  fbi_cd                 232015 non-null  object 
 11  latitude               232015 non-null  float64
 12  longitude              232015 non-null  float64
 13  month                  232015 non-null  object 
 14  week_day               232015 non-nu

In [ ]:
del df["date_of_occurrence"]
del df["arrest"]
del df["iucr"]
del df["latitude"]
del df["longitude"]

# **One hot encoding for features with many variables**
In this i will pick up top most freqquencty occuring labels from each feature and for the rest of the labels if they occur in the data would have their complete row as zero and they will not contain a column of their own


## Counting columns and unique values for each column before encoding

[onehotencoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [ ]:
orig_rows, orig_col= df.shape
print(" Rows",orig_rows,"\n","Columns",orig_col)

 Rows 232015 
 Columns 11


In [ ]:
for column in df.columns:
    print(column, ": ", df[column].nunique(), "| ", df[column].nunique()/orig_rows*100,"%")
print("Total rows: ", orig_rows)

block :  26988 |  11.632006551300563 %
primary_description :  32 |  0.013792211710449755 %
secondary_description :  427 |  0.1840398250113139 %
location_description :  162 |  0.06982307178415188 %
domestic :  2 |  0.0008620132319031097 %
beat :  274 |  0.11809581277072603 %
ward :  50 |  0.021550330797577743 %
fbi_cd :  26 |  0.011206172014740426 %
month :  12 |  0.005172079391418658 %
week_day :  7 |  0.003017046311660884 %
hour :  24 |  0.010344158782837317 %
Total rows:  232015


In [ ]:
df.columns

Index(['block', 'primary_description', 'secondary_description',
       'location_description', 'domestic', 'beat', 'ward', 'fbi_cd', 'month',
       'week_day', 'hour'],
      dtype='object')

# **Data For Predicting Secondary Type**

# **Which values to pick?**

, block, beat, location_description, ,domestic, beat, ward, fbi_cd, month, week_day, hour


**Top Values :**

* top 12 **primary_description**
description
* top 20 wards
    * top 30 **blocks** from top wards
    * top 2 **beats** from top wards
* top 30 **location description**
* top 10 **FBI code**

**Others**

* domestic :  2
* month :  12
* week_day :  7
* hour :  24



# **Encoder Function**

This function accepts:
* **column_name**: The column to encode
* **top_values**: List of labels to encode

In [ ]:
def encoder_function(column_name, labels):
    """
    This function accepts:
    column_name: The column to encode
    labels: List of values to encode
    """
    for category in labels:
        new_col= column_name + "_"+ category
        df[new_col]= np.where(df[column_name] == category, 1, 0)

# **Getting internal top 10 values function**

In [ ]:
def list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count):
    """
    This function accepts:
    column_name: The name of the outer level column
    outer_top: (List) top values of outer column
    no_of_top_topick: Number of top values to pick from inner column
    Column_to_count: Name of column from which you want to pick top values of
    """
    top_values=[]
    for value in outer_top:
        filt= (df[column_name] == value )
        top_values.extend([x for x in df[filt][Column_to_count].value_counts(ascending= False).head(no_of_top_topick).index])
    return top_values

# **Top Labels and their encoding**

## **Top 10 primary description**

In [ ]:
n_values= 12
top_prim_des= [x for x in df["primary_description"].value_counts(ascending= False).head(n_values).index]
top_values = top_prim_des.copy()

### **Encoding Top Primary Description values**

In [ ]:
column_name= "primary_description"
encoder_function(column_name, top_values)

In [ ]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0


## **Top Wards**

In [ ]:
n_values= 20
column_= "ward"
top_wards= [x for x in df[column_].value_counts(ascending= False).head(n_values).index]
top_values = list(map(str,top_wards))

### **Encoding Wards**

In [ ]:
column_name= "ward"
encoder_function(column_name, top_values)

In [ ]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,ward_5.0,ward_25.0,ward_1.0
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top Blocks from top wards**

In [ ]:
# Getting list of blocks that occur in top wards
# to pass
column_name= "ward" # The name of the outer level column
outer_top= top_wards # Top values of outer column
no_of_top_topick =10 #these are how many number of values to pick for the inner top values
Column_to_count= "block" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))
print(top_values)

['067XX S EAST END AVE', '050XX S MICHIGAN AVE', '006XX W HARRISON ST', '074XX S COLFAX AVE', '079XX S DREXEL AVE', '007XX E 79TH ST', '002XX W 103RD ST', '015XX N CLYBOURN AVE', '071XX S ASHLAND AVE', '002XX E 121ST PL', '075XX S STONY ISLAND AVE', '063XX S ASHLAND AVE', '002XX N PINE AVE', '056XX W MADISON ST', '039XX W VAN BUREN ST', '054XX W CHICAGO AVE', '075XX S ASHLAND AVE', '062XX S ASHLAND AVE', '0000X S HALSTED ST', '115XX S HALSTED ST', '069XX S CLYDE AVE', '054XX S WENTWORTH AVE', '039XX W ROOSEVELT RD', '083XX S HOLLAND RD', '016XX S HOMAN AVE', '007XX E 111TH ST', '015XX N DAMEN AVE', '047XX W NORTH AVE', '007XX S CICERO AVE', '017XX E 67TH ST', '016XX W DIVISION ST', '013XX W 76TH ST', '014XX W 63RD ST', '079XX S LAFAYETTE AVE', '081XX S COTTAGE GROVE AVE', '132XX S LANGLEY AVE', '002XX W 47TH ST', '023XX N MILWAUKEE AVE', '054XX W MADISON ST', '068XX S PERRY AVE', '005XX N STATE ST', '006XX S STATE ST', '010XX S CANAL ST', '066XX S STONY ISLAND AVE', '023XX E 79TH ST', 

### **Encoding Blocks**

In [ ]:
column_name= "block"
encoder_function(column_name, top_values)

In [ ]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,block_061XX S DR MARTIN LUTHER KING JR DR,block_013XX N PAULINA ST,block_013XX N HUDSON AVE,block_037XX W CHICAGO AVE,block_042XX S MICHIGAN AVE,block_063XX S KEDZIE AVE,block_111XX S STATE ST,block_011XX S CLARK ST,block_079XX S WABASH AVE,block_094XX S ASHLAND AVE,block_065XX S DR MARTIN LUTHER KING JR DR,block_076XX S STEWART AVE,block_001XX W CERMAK RD,block_0000X S STATE ST,block_062XX S WESTERN AVE,block_051XX W NORTH AVE,block_039XX W GLADYS AVE,block_079XX S HALSTED ST,block_123XX S WALLACE ST,block_077XX S SOUTH SHORE DR,block_082XX S MARYLAND AVE,block_098XX S HALSTED ST,block_006XX N MICHIGAN AVE,block_006XX S CENTRAL AVE,block_006XX W 63RD ST,block_109XX S DOTY AVE W,block_005XX N CENTRAL AVE,block_039XX W JACKSON BLVD,block_099XX S HALSTED ST,block_0000X W 87TH ST,block_098XX S STATE ST,block_063XX S DR MARTIN LUTHER KING JR DR,block_0000X E GRAND AVE,block_130XX S ELLIS AVE,block_070XX S COTTAGE GROVE AVE,block_005XX E 79TH ST,block_111XX S MICHIGAN AVE,block_082XX S INGLESIDE AVE,block_021XX E 71ST ST,block_078XX S HALSTED ST
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top beats from top wards**

In [ ]:
# Getting list of blocks that occur in top wards
# to pass
column_name= "ward" # The name of the outer level column
outer_top= top_wards # Top values of outer column
no_of_top_topick =2 #these are how many number of values to pick for the inner top values
Column_to_count= "beat" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))
print(top_values)

[131, 522, 524, 1424, 533, 1431, 414, 1533, 421, 423, 1834, 312, 313, 712, 713, 331, 332, 1234, 213, 1112, 1113, 222, 1121, 612, 613, 2533, 1511, 621, 622, 111, 623, 624, 1522, 1011, 1524, 631, 123, 124, 1021, 511]


#### **Encoding Top Beats**

In [ ]:
column_name= "beat"
top_values= list(map(str,top_values))
encoder_function(column_name, top_values)

In [ ]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,beat_131,beat_522,beat_524,beat_1424,beat_533,beat_1431,beat_414,beat_1533,beat_421,beat_423,beat_1834,beat_312,beat_313,beat_712,beat_713,beat_331,beat_332,beat_1234,beat_213,beat_1112,beat_1113,beat_222,beat_1121,beat_612,beat_613,beat_2533,beat_1511,beat_621,beat_622,beat_111,beat_623,beat_624,beat_1522,beat_1011,beat_1524,beat_631,beat_123,beat_124,beat_1021,beat_511
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top Locations**

In [ ]:
top_prim_des= [x for x in df["location_description"].value_counts(ascending= False).head(50).index]
top_values = top_prim_des.copy()
top_values

['STREET',
 'RESIDENCE',
 'APARTMENT',
 'SIDEWALK',
 'OTHER',
 'SMALL RETAIL STORE',
 'RESTAURANT',
 'PARKING LOT/GARAGE(NON.RESID.)',
 'ALLEY',
 'DEPARTMENT STORE',
 'VEHICLE NON-COMMERCIAL',
 'GAS STATION',
 'RESIDENCE PORCH/HALLWAY',
 'RESIDENTIAL YARD (FRONT/BACK)',
 'GROCERY FOOD STORE',
 'RESIDENCE-GARAGE',
 'CONVENIENCE STORE',
 'OTHER (SPECIFY)',
 'CTA TRAIN',
 'SCHOOL, PUBLIC, BUILDING',
 'BAR OR TAVERN',
 'COMMERCIAL / BUSINESS OFFICE',
 'PARK PROPERTY',
 'PARKING LOT / GARAGE (NON RESIDENTIAL)',
 'DRUG STORE',
 'RESIDENCE - PORCH / HALLWAY',
 'RESIDENCE - YARD (FRONT / BACK)',
 'CTA BUS',
 'CTA STATION',
 'BANK',
 'HOSPITAL BUILDING/GROUNDS',
 'HOTEL/MOTEL',
 'CTA PLATFORM',
 'CHA APARTMENT',
 'RESIDENCE - GARAGE',
 'NURSING HOME/RETIREMENT HOME',
 'DRIVEWAY - RESIDENTIAL',
 'SCHOOL, PUBLIC, GROUNDS',
 'CTA BUS STOP',
 'POLICE FACILITY/VEH PARKING LOT',
 'VACANT LOT/LAND',
 'CHA PARKING LOT/GROUNDS',
 'TAVERN/LIQUOR STORE',
 'CURRENCY EXCHANGE',
 'SCHOOL, PRIVATE, BUILDING',

### **Encoding Top Locations**

In [ ]:
column_name= "location_description"
encoder_function(column_name, top_values)

In [ ]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,location_description_VEHICLE NON-COMMERCIAL,location_description_GAS STATION,location_description_RESIDENCE PORCH/HALLWAY,location_description_RESIDENTIAL YARD (FRONT/BACK),location_description_GROCERY FOOD STORE,location_description_RESIDENCE-GARAGE,location_description_CONVENIENCE STORE,location_description_OTHER (SPECIFY),location_description_CTA TRAIN,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,location_description_HOSPITAL BUILDING/GROUNDS,location_description_HOTEL/MOTEL,location_description_CTA PLATFORM,location_description_CHA APARTMENT,location_description_RESIDENCE - GARAGE,location_description_NURSING HOME/RETIREMENT HOME,location_description_DRIVEWAY - RESIDENTIAL,"location_description_SCHOOL, PUBLIC, GROUNDS",location_description_CTA BUS STOP,location_description_POLICE FACILITY/VEH PARKING LOT,location_description_VACANT LOT/LAND,location_description_CHA PARKING LOT/GROUNDS,location_description_TAVERN/LIQUOR STORE,location_description_CURRENCY EXCHANGE,"location_description_SCHOOL, PRIVATE, BUILDING",location_description_CHURCH/SYNAGOGUE/PLACE OF WORSHIP,location_description_WAREHOUSE,location_description_GOVERNMENT BUILDING/PROPERTY,location_description_ATHLETIC CLUB,location_description_CONSTRUCTION SITE
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top FBI Code**

In [ ]:
top_prim_des= [x for x in df["fbi_cd"].value_counts(ascending= False).head(10).index]
top_values = top_prim_des.copy()
top_values

['06', '08B', '14', '26', '08A', '11', '18', '05', '07', '03']

### **Encoding top FBI codes**

In [ ]:
column_name= "fbi_cd"
encoder_function(column_name, top_values)

In [ ]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,location_description_HOSPITAL BUILDING/GROUNDS,location_description_HOTEL/MOTEL,location_description_CTA PLATFORM,location_description_CHA APARTMENT,location_description_RESIDENCE - GARAGE,location_description_NURSING HOME/RETIREMENT HOME,location_description_DRIVEWAY - RESIDENTIAL,"location_description_SCHOOL, PUBLIC, GROUNDS",location_description_CTA BUS STOP,location_description_POLICE FACILITY/VEH PARKING LOT,location_description_VACANT LOT/LAND,location_description_CHA PARKING LOT/GROUNDS,location_description_TAVERN/LIQUOR STORE,location_description_CURRENCY EXCHANGE,"location_description_SCHOOL, PRIVATE, BUILDING",location_description_CHURCH/SYNAGOGUE/PLACE OF WORSHIP,location_description_WAREHOUSE,location_description_GOVERNMENT BUILDING/PROPERTY,location_description_ATHLETIC CLUB,location_description_CONSTRUCTION SITE,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


# **Label Encoding of other  columns**

## **Encoding domestic Column**

In [ ]:
column_name= "domestic"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,location_description_HOSPITAL BUILDING/GROUNDS,location_description_HOTEL/MOTEL,location_description_CTA PLATFORM,location_description_CHA APARTMENT,location_description_RESIDENCE - GARAGE,location_description_NURSING HOME/RETIREMENT HOME,location_description_DRIVEWAY - RESIDENTIAL,"location_description_SCHOOL, PUBLIC, GROUNDS",location_description_CTA BUS STOP,location_description_POLICE FACILITY/VEH PARKING LOT,location_description_VACANT LOT/LAND,location_description_CHA PARKING LOT/GROUNDS,location_description_TAVERN/LIQUOR STORE,location_description_CURRENCY EXCHANGE,"location_description_SCHOOL, PRIVATE, BUILDING",location_description_CHURCH/SYNAGOGUE/PLACE OF WORSHIP,location_description_WAREHOUSE,location_description_GOVERNMENT BUILDING/PROPERTY,location_description_ATHLETIC CLUB,location_description_CONSTRUCTION SITE,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,domestic_N,domestic_Y
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


## **Encoding month Column**

In [ ]:
column_name= "month"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,location_description_RESIDENCE - GARAGE,location_description_NURSING HOME/RETIREMENT HOME,location_description_DRIVEWAY - RESIDENTIAL,"location_description_SCHOOL, PUBLIC, GROUNDS",location_description_CTA BUS STOP,location_description_POLICE FACILITY/VEH PARKING LOT,location_description_VACANT LOT/LAND,location_description_CHA PARKING LOT/GROUNDS,location_description_TAVERN/LIQUOR STORE,location_description_CURRENCY EXCHANGE,"location_description_SCHOOL, PRIVATE, BUILDING",location_description_CHURCH/SYNAGOGUE/PLACE OF WORSHIP,location_description_WAREHOUSE,location_description_GOVERNMENT BUILDING/PROPERTY,location_description_ATHLETIC CLUB,location_description_CONSTRUCTION SITE,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,domestic_N,domestic_Y,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0


## **Encoding weekday Column**

In [ ]:
column_name= "week_day"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,location_description_CHA PARKING LOT/GROUNDS,location_description_TAVERN/LIQUOR STORE,location_description_CURRENCY EXCHANGE,"location_description_SCHOOL, PRIVATE, BUILDING",location_description_CHURCH/SYNAGOGUE/PLACE OF WORSHIP,location_description_WAREHOUSE,location_description_GOVERNMENT BUILDING/PROPERTY,location_description_ATHLETIC CLUB,location_description_CONSTRUCTION SITE,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,domestic_N,domestic_Y,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,week_day_Wednesday
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## **Encoding hour Column**

In [ ]:
column_name= "hour"

labels = list(df[column_name].unique())

labels= list(map(str,labels))
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,...,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,week_day_Wednesday,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_0,hour_1,hour_2,hour_3,hour_4
0,061XX W GRAND AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,2512,36,08B,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,ASSAULT,SIMPLE,CTA BUS,N,1624,45,08A,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Deleting Label Columns**

In [ ]:
data.columns

Index(['date_of_occurrence', 'block', 'iucr', 'primary_description',
       'secondary_description', 'location_description', 'arrest', 'domestic',
       'beat', 'ward', 'fbi_cd', 'latitude', 'longitude', 'month', 'week_day',
       'hour'],
      dtype='object')

In [ ]:
del df["block"]
del df["primary_description"]
del df["location_description"]
del df["domestic"]
del df["beat"]
del df["fbi_cd"]
del df["month"]
del df["week_day"]
del df["hour"]
del df["ward"]

In [ ]:
df.head(2)

,secondary_description,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,primary_description_WEAPONS VIOLATION,primary_description_CRIMINAL TRESPASS,ward_42.0,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_17.0,ward_21.0,ward_8.0,ward_20.0,ward_16.0,ward_37.0,ward_9.0,ward_7.0,ward_3.0,ward_34.0,ward_4.0,ward_29.0,ward_5.0,ward_25.0,ward_1.0,block_067XX S EAST END AVE,block_050XX S MICHIGAN AVE,block_006XX W HARRISON ST,block_074XX S COLFAX AVE,block_079XX S DREXEL AVE,block_007XX E 79TH ST,block_002XX W 103RD ST,...,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,week_day_Wednesday,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_0,hour_1,hour_2,hour_3,hour_4
0,DOMESTIC BATTERY SIMPLE,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SIMPLE,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Saving Secondary Description data**

In [ ]:
df.to_csv("sec_des_data.csv")